In [13]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import datetime
from decimal import Decimal

import warnings
warnings.filterwarnings("ignore")

# Chargement des données à partir du fichier Excel
file_path =  r"C:\Users\Administrateur\Desktop\Dashboardv001\inputcons\BD.xlsx"

df = pd.read_excel(file_path)

# Obtenir la liste des mois uniques dans la colonne 'Mois'
mois_list = df['Mois'].unique()

# Obtenir la liste des années uniques dans la colonne 'Année'
annee_list = df['Années'].unique()

# Obtenir la liste des années uniques dans la colonne 'Année'
categorie_list = df['Catégorie'].unique()

# Obtenir la liste des années uniques dans la colonne 'Année'
sous_categorie_list = df['Sous-catégorie'].unique()

# Fonctions pour générer les visualisations des KPIs

#########    1   ########

def generate_pie_chart_weight_on_revenue(filtered_df):
    df_category_revenue = filtered_df.groupby('Catégorie')['Total HT'].sum().reset_index()
    total_revenue = df_category_revenue['Total HT'].sum()

    df_category_revenue['Poids'] = df_category_revenue['Total HT'] / total_revenue
    df_category_revenue['Valeur Absolue'] = df_category_revenue['Total HT'] * 0.75  # 75% of Total HT

    # Trouver l'indice de la part la plus petite
    min_index = df_category_revenue['Poids'].idxmin()
    explode = [0.1 if i == min_index else 0 for i in range(len(df_category_revenue))]

    fig = go.Figure(go.Pie(
        labels=df_category_revenue['Catégorie'],
        values=df_category_revenue['Poids'],
        textinfo='label+percent',
        hovertemplate='<b>%{label}</b><br>Poids : %{percent:.2%}<br>Valeur Absolue : %{customdata} FCFA',
        customdata=df_category_revenue['Valeur Absolue'],
        marker=dict(
            colors=['#FF5733', '#FFC300', '#36D7B7', '#3C40C6', '#27AE60', '#F39C12', '#9B59B6', '#D4AC0D', '#E74C3C', '#3498DB']
        ),
        hole=0.4,
        sort=False,
        pull=explode  # Appliquer la fonction d'explosion
    ))

    fig.update_layout()

    return fig


#########    2    ########
def generate_treemap_item_subcategory(filtered_df):
    df = filtered_df.copy()
    total_relative_quantity = df['Quantité Relative'].sum()
    df['PctRelative'] = df['Quantité Relative'] / total_relative_quantity * 100  # Calcul du pourcentage

    fig = px.treemap(df, path=['Catégorie', 'Sous-catégorie', 'Item'],
                     values='PctRelative',  # Utilisation des pourcentages calculés
                     custom_data=['PctRelative'],  # Stockage des pourcentages dans les données personnalisées
                     color='Sous-catégorie')

    fig.update_layout()

    fig.update_traces(
        hovertemplate='<b>%{label}</b><br>Poids: %{customdata[0]:.2f}%',  # Format en pourcentage avec 2 décimales
        textinfo='label+percent entry'  # Affiche le label de la catégorie et le pourcentage
    )

    return fig

#########    3    ########
def generate_sunburst_item_category(filtered_df):
    df = filtered_df.copy()
    total_relative_quantity = df['Quantité Relative'].sum()
    df['PctRelative'] = df['Quantité Relative'] / total_relative_quantity * 100

    fig = px.sunburst(df, path=['Catégorie', 'Item'], 
                      values='PctRelative',
                      custom_data=['PctRelative'],
                      title='Distribution des Items par Catégorie', color='Catégorie')

    fig.update_traces(
        hovertemplate='<b>%{label}</b><br>Poids: %{customdata[0]:.2f}%',
        textinfo='label+percent entry'
    )
    
    # Personnalisation du style du titre
    fig.update_layout()

    return fig


#########    4   ########
def generate_sunburst_subcategory_within_category(filtered_df):
    df = filtered_df.copy()
    total_relative_quantity = df['Quantité Relative'].sum()
    df['PctRelative'] = df['Quantité Relative'] / total_relative_quantity * 100  # Calcul du pourcentage

    fig = px.sunburst(df, path=['Catégorie', 'Sous-catégorie'], 
                      values='PctRelative',  # Utilisation des pourcentages calculés
                      custom_data=['PctRelative']  # Stockage des pourcentages dans les données personnalisées
                    #   title='Poids de chaque sous-catégorie au sein de sa catégorie'
                      )

    fig.update_traces(
        hovertemplate='<b>%{label}</b><br>Poids: %{customdata[0]:.2f}%',  # Format en pourcentage avec 2 décimales
        textinfo='label+percent entry'  # Affiche le label de la sous-catégorie et le pourcentage
    )

    # Personnalisation du style du titre
    fig.update_layout( )
    return fig


#########    5    ########
def generate_bar_weight_on_revenue(filtered_df):
    df_category_revenue = filtered_df.groupby(['Catégorie', 'Sous-catégorie'])['Total HT'].sum().reset_index()
    total_revenue = df_category_revenue['Total HT'].sum()
    
    df_category_revenue['Poids'] = df_category_revenue['Total HT'] / total_revenue * 100  # Calcul du pourcentage

    fig = px.bar(df_category_revenue, x='Catégorie', y='Poids', 
                 title='Poids de chaque catégorie et sous-catégorie sur le chiffre d\'affaires global')
    
    fig.update_traces(
        texttemplate='%{y:.2f}%',  # Format en pourcentage avec 2 décimales
        textposition='outside'  # Affiche les pourcentages à l'extérieur des barres
    )

     # Personnalisation du style du titre
    fig.update_layout()
    
    return fig


#########   6   ########
def generate_box_category_revenue(filtered_df):
    fig = px.box(filtered_df, x='Catégorie', y='Total HT')
    fig.update_layout()
    return fig


#########    7    ########
def generate_box_total_revenue(filtered_df):
    fig = px.box(filtered_df, y='Total HT')
    
    fig.update_layout()
    return fig


# Création du tableau de bord
app = dash.Dash(__name__)

# Ajustement de la taille des graphiques Sunburst
sunburst_height = 200

app.layout = html.Div([
    html.Link(
        rel='stylesheet',
        href='https://adminlte.io/themes/v3/plugins/fontawesome-free/css/all.min.css'),    
    html.Link(
        rel='stylesheet',
        href='https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css'),
    html.Link(
        rel='stylesheet',
        href='https://adminlte.io/themes/v3/dist/css/adminlte.min.css?v=3.2.0'),

    html.Div([
        html.Div([
            html.Div([
                html.Div([
                    html.H1("Dashboard d'analyse des données", className="m-0")
                ], className="col-sm-8"),
                html.Div([
                    html.Ol([
                        html.Li(id='current-time', className="breadcrumb-item active")
                    ], className="breadcrumb float-sm-right")
                ], className="col-sm-4")
            ], className="row mb-2"),
            html.Div([
                html.Div([
                    dcc.Dropdown(id='year-dropdown', options=[{'label': str(annee), 'value': annee} for annee in annee_list],
                                value=None, placeholder="Sélectionnez les années", multi=True)
                ], className='col-md-3'),

                html.Div([
                    dcc.Dropdown(id='month-dropdown', options=[{'label': mois, 'value': mois} for mois in mois_list],
                                value=None, placeholder="Sélectionnez les mois", multi=True)
                ], className='col-md-3'),
                    
                html.Div([
                    dcc.Dropdown(id='categorie-dropdown', options=[{'label': str(categorie), 'value': categorie} for categorie in categorie_list],
                                value=None, placeholder="Sélectionnez les catégories", multi=True)
                ], className='col-md-3'),

                html.Div([
                    dcc.Dropdown(id='sous-categorie-dropdown', options=[{'label': str(sous_categorie), 'value': sous_categorie} for sous_categorie in sous_categorie_list],
                                value=None, placeholder="Sélectionnez les sous catégorie", multi=True)
                ], className='col-md-3'),

            ], className='row mb-3'),

            html.Div(id='revenue-summary')

        ], className="container-fluid")
    ], className="content-header mb-4 pb-1", style={'background-color': '#c2c2c3'} ),

    html.Section([
        html.Div([
            html.Div([
                html.Div(id='visualizations-container'),
            ], className="container-fluid")
        ], className="row"),
    ], className="content"),

    dcc.Interval(id='interval-component', interval=1000, n_intervals=0),  # Rafraîchissement toutes les secondes
], className='content-wrapper', style={'margin-left': '0px', 'min-height': '100vh'})

@app.callback(
    Output('sous-categorie-dropdown', 'options'),
    Input('categorie-dropdown', 'value')
)
def update_sous_categorie_dropdown(selected_categories):
    if selected_categories is None:
        return []
    
    filtered_df = df[df['Catégorie'].isin(selected_categories)]
    sous_categorie_options = [{'label': sous_categorie, 'value': sous_categorie} for sous_categorie in filtered_df['Sous-catégorie'].unique()]
    return sous_categorie_options

@app.callback(
    Output('revenue-summary', 'children'),
    Input('month-dropdown', 'value'),
    Input('year-dropdown', 'value'),
    Input('categorie-dropdown', 'value'),
    Input('sous-categorie-dropdown', 'value'),
    Input('interval-component', 'n_intervals')
)
def update_revenue_summary(selected_months, selected_years, selected_categories, selected_sous_categories, n_intervals):
    if selected_months is None or selected_years is None:
        return html.Div()

    filtered_df = df[df['Années'].isin(selected_years)]
    filtered_df = filtered_df[df['Mois'].isin(selected_months)]
    
    if selected_categories:
        filtered_df = filtered_df[filtered_df['Catégorie'].isin(selected_categories)]
    
    if selected_sous_categories:
        filtered_df = filtered_df[filtered_df['Sous-catégorie'].isin(selected_sous_categories)]

    if filtered_df.empty:
        return html.Div("")

    total_revenue = filtered_df['Total HT'].sum()
    formatted_total_revenue = '{:,.2f}'.format(total_revenue).replace(',', ' ').replace('.', ',') + " FCFA"

    selected_month_names = ', '.join(map(str, selected_months))
    selected_year_names = ', '.join(map(str, selected_years))
    
    if selected_categories:
        selected_categorie_names = ', '.join(selected_categories)
        if selected_sous_categories:
            selected_sous_categorie_names = ', '.join(selected_sous_categories)
            formatted_message = f"Le chiffre d'affaires de(s) sous-catégorie(s) {selected_sous_categorie_names} de(s) catégorie(s) {selected_categorie_names} du mois de {selected_month_names} de l'année {selected_year_names}"
        else:
            formatted_message = f"Le chiffre d'affaires de(s) catégorie(s) {selected_categorie_names} du mois de {selected_month_names} de l'année {selected_year_names}"
    else:
        formatted_message = f"Le chiffre d'affaires du mois de {selected_month_names} de l'année {selected_year_names}"

    return html.Div([
        html.Div([
            html.Div([
                html.H3(f"{formatted_total_revenue}"),
                html.P(f"{formatted_message}", className="mb-0"),
            ], className="inner p-2"),
        ], className="small-box bg-info col-md-12 col-12")
    ], className="")


@app.callback(
    Output('current-time', 'children'),
    Input('interval-component', 'n_intervals')
)
def update_current_time(n_intervals):
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"Heure actuelle : {current_time}"

@app.callback(
    Output('visualizations-container', 'children'),
    Input('month-dropdown', 'value'),
    Input('year-dropdown', 'value'),
    Input('categorie-dropdown', 'value'),
    Input('sous-categorie-dropdown', 'value')
)
def update_visualizations(selected_months, selected_years, selected_categories, selected_sous_categories):
    if selected_months is None or selected_years is None:
        return html.Div()

    filtered_df = df[df['Années'].isin(selected_years)]
    filtered_df = filtered_df[df['Mois'].isin(selected_months)]
    
    if selected_categories:
        filtered_df = filtered_df[filtered_df['Catégorie'].isin(selected_categories)]
    
    if selected_sous_categories:
        filtered_df = filtered_df[filtered_df['Sous-catégorie'].isin(selected_sous_categories)]

    if filtered_df.empty:
        return html.Div([
            html.Div([
                html.Div([
                    html.Div([
                        html.H1("Aucune donnée disponible pour les sélections faites."),
                    ], className="inner p-2"),
                ])
            ], className="small-box bg-danger")
        ], className="col-lg-12 col-12")
    
    # Utilisation des différentes fonctions de génération de graphiques
    fig_pie_chart_weight_on_revenue = generate_pie_chart_weight_on_revenue(filtered_df)
    fig_treemap_item_subcategory = generate_treemap_item_subcategory(filtered_df)
    fig_sunburst_item_category = generate_sunburst_item_category(filtered_df)
    fig_sunburst_subcategory_within_category = generate_sunburst_subcategory_within_category(filtered_df)
    fig_bar_weight_on_revenue = generate_bar_weight_on_revenue(filtered_df)
    fig_box_category_revenue = generate_box_category_revenue(filtered_df)
    fig_box_total_revenue = generate_box_total_revenue(filtered_df)

    return html.Div([
            html.Div([
                html.Div([
                    html.Div([
                        html.H3("Poids de chaque catégorie sur le chiffre d'affaires global", className="card-title")
                    ], className="card-header"),
                    html.Div([
                        html.Div([
                            html.Div(dcc.Graph(figure=fig_pie_chart_weight_on_revenue.update_layout(margin=dict(t=0, b=0, l=0, r=0)))),
                        ], className="card-body pad table-responsive p-0")
                    ], className="card-body")
                ], className="card card-primary card-outline")
            ], className="col-md-6"),

            html.Div([
                html.Div([
                    html.Div([
                        html.H3("Poids de chaque Item dans sa sous-catégorie", className="card-title")
                    ], className="card-header"),
                    html.Div([
                        html.Div([
                            html.Div(dcc.Graph(figure=fig_treemap_item_subcategory.update_layout(margin=dict(t=0, b=0, l=0, r=0)))),
                        ], className="card-body pad table-responsive p-0")
                    ], className="card-body")
                ], className="card card-primary card-outline")
            ], className="col-md-6"),

            html.Div([
                html.Div([
                    html.Div([
                        html.H3("Distribution des Items par Catégorie", className="card-title")
                    ], className="card-header"),
                    html.Div([
                        html.Div([
                            html.Div(dcc.Graph(figure=fig_sunburst_item_category.update_layout(margin=dict(t=0, b=0, l=0, r=0)))),
                        ], className="card-body pad table-responsive p-0")
                    ], className="card-body")
                ], className="card card-primary card-outline")
            ], className="col-md-6"),

            html.Div([
                html.Div([
                    html.Div([
                        html.H3("Poids de chaque sous-catégorie au sein de sa catégorie", className="card-title")
                    ], className="card-header"),
                    html.Div([
                        html.Div([
                            html.Div(dcc.Graph(figure=fig_sunburst_subcategory_within_category.update_layout(margin=dict(t=0, b=0, l=0, r=0)))),
                        ], className="card-body pad table-responsive p-0")
                    ], className="card-body")
                ], className="card card-primary card-outline")
            ], className="col-md-6"),

            html.Div([
                html.Div([
                    html.Div([
                        html.H3("Poids de chaque catégorie et sous-catégorie sur le chiffre d'affaires global", className="card-title")
                    ], className="card-header"),
                    html.Div([
                        html.Div([
                            html.Div(dcc.Graph(figure=fig_bar_weight_on_revenue.update_layout(margin=dict(t=0, b=0, l=0, r=0)))),
                        ], className="card-body pad table-responsive p-0")
                    ], className="card-body")
                ], className="card card-primary card-outline")
            ], className="col-md-4"),

            html.Div([
                html.Div([
                    html.Div([
                        html.H3("Chiffre d'affaires global de chaque catégorie", className="card-title")
                    ], className="card-header"),
                    html.Div([
                        html.Div([
                            html.Div(dcc.Graph(figure=fig_box_category_revenue.update_layout(margin=dict(t=0, b=0, l=0, r=0)))),
                        ], className="card-body pad table-responsive p-0")
                    ], className="card-body")
                ], className="card card-primary card-outline")
            ], className="col-md-4"),

            html.Div([
                html.Div([
                    html.Div([
                        html.H3("Chiffre d'affaires global de chaque catégorie", className="card-title")
                    ], className="card-header"),
                    html.Div([
                        html.Div([
                            html.Div(dcc.Graph(figure=fig_box_total_revenue.update_layout(margin=dict(t=0, b=0, l=0, r=0)))),
                        ], className="card-body pad table-responsive p-0")
                    ], className="card-body")
                ], className="card card-primary card-outline")
            ], className="col-md-4")

        ], className="row")

if __name__ == '__main__':
    app.run_server(debug=True, port=5514)
